# 1. Data prep & TF input pipeline for the Arabic **letters** model

This block mounts Google Drive, locates and extracts the dataset zip, enumerates classes, and builds an efficient TensorFlow `tf.data` pipeline that turns raw WAV files into normalized log-mel spectrogram tensors ready for model training.

- **Mount & extract:** Mounts Drive, finds `arabic_letters_dataset.zip`, extracts it to `/content/hijaiyah_data`, and lists top-level contents for a quick sanity check.  
- **Dataset layout & counts:** Sets `BASE_PATH = "/content/hijaiyah_data/Dataset Telkom 16KHz"`. Iterates through class folders (`"1"` … `"28"`), counts `.wav` files per class, and prints totals—useful for verifying class balance.  
- **Class mapping:** Defines a stable mapping from numeric folder IDs (`"1"`–`"28"`) to ordered labels (`01_A`, `02_Ba`, …, `28_Ya`). Builds parallel arrays `file_paths` and integer `labels` (0-indexed) and reports per-class sample counts plus a few sample (path, label) pairs.  
- **Train/valid split & shuffling:** Uses `np.random.default_rng(SEED=42)` to shuffle all examples deterministically, then performs an 80/20 split into `train_*` and `valid_*`.  
- **Audio → log-mel features:**  
  - **I/O:** `load_wav_16k_mono()` reads each file and decodes to mono at **16 kHz** (`SAMPLE_RATE=16000`).  
  - **STFT/Mel:** `FRAME_LENGTH=512`, `FRAME_STEP=256`, projects the magnitude spectrogram to **128 mel bins** (`NUM_MELS=128`) over **80–7600 Hz**.  
  - **Log-scale & min–max norm:** Applies `log(mel+1e-6)` then per-example min–max normalization.  
  - **Shape standardization:** Adds a channel dim and pads/resizes to **(time=61, mel=128, channels=1)** so all inputs share a fixed size.  
- **tf.data pipelines:**  
  - **Training:** Builds `train_ds` by mapping `preprocess` → `cache()` → `shuffle(1000)` → `batch(16)` → `prefetch(AUTOTUNE)` for throughput.  
  - **Validation:** Builds `valid_ds` similarly (without shuffle).  
  - Prints one batch’s shapes to confirm tensors are `(batch, 61, 128, 1)` and labels `(batch,)`.  

**Key hyperparameters:** `SAMPLE_RATE=16000`, `FRAME_LENGTH=512`, `FRAME_STEP=256`, `NUM_MELS=128`, `LOWER_HZ=80`, `UPPER_HZ=7600`, `TARGET_TIME_FRAMES=61`, `TARGET_MEL_BINS=128`, `BATCH_SIZE=16`, `SEED=42`.  

**Purpose:** End-to-end data preparation—from Drive zip to balanced, batched, prefetched log-mel tensors—for training an Arabic letters classifier across 28 classes.


In [ ]:
from google.colab import drive
from google.colab import files
from IPython.display import Audio, display

import os
import glob
import json
import uuid
import subprocess

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import Sequential, callbacks, metrics
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
drive.mount('/content/drive')



BASE_PATH = "/content/drive/MyDrive/QariAI/arabic_letters_dataset"

print("Dataset path:", BASE_PATH)
print("Number of subfolders:", len(os.listdir(BASE_PATH)))
print("First few subfolders:", sorted(os.listdir(BASE_PATH))[:10])


In [ ]:
counts = {}
for d in sorted(os.listdir(BASE_PATH)):
    dpath = os.path.join(BASE_PATH, d)
    if os.path.isdir(dpath):
        wavs = glob.glob(os.path.join(dpath, "*.wav"))
        counts[d] = len(wavs)

print("Total classes found:", len(counts))
for k, v in counts.items():
    print(f"Class {k} -> {v} files")


In [ ]:
CLASS_ID_TO_NAME = {
    "1":"01_A","2":"02_Ba","3":"03_Ta","4":"04_Tsa","5":"05_Ja","6":"06_Hha","7":"07_Kha",
    "8":"08_Da","9":"09_Dza","10":"10_Ro","11":"11_Za","12":"12_Sa","13":"13_Sya","14":"14_Sho",
    "15":"15_Dho","16":"16_Tho","17":"17_Zho","18":"18_Ain","19":"19_Gho","20":"20_Fa","21":"21_Qo",
    "22":"22_Ka","23":"23_La","24":"24_Ma","25":"25_Na","26":"26_Ha","27":"27_Wa","28":"28_Ya"
}

CLASS_IDS = [str(i) for i in range(1, 29)]
CLASSES = [CLASS_ID_TO_NAME[cid] for cid in CLASS_IDS]
CLASS_TO_INDEX = {name: i for i, name in enumerate(CLASSES)}
NUM_CLASSES = len(CLASSES)

print("Classes (ordered):", CLASSES)
print("NUM_CLASSES:", NUM_CLASSES)

file_paths, labels = [], []
per_class_counts = {}

for idx, cid in enumerate(CLASS_IDS):
    class_dir = os.path.join(BASE_PATH, cid)
    wavs = sorted(glob.glob(os.path.join(class_dir, "*.wav")))
    file_paths.extend(wavs)
    labels.extend([idx] * len(wavs))
    per_class_counts[CLASSES[idx]] = len(wavs)

file_paths = np.array(file_paths)
labels = np.array(labels, dtype=np.int64)

print(f"\nTotal audio files: {len(file_paths)}")
print("Per-class counts:")
for k in CLASSES:
    print(f"{k:>7s} -> {per_class_counts.get(k, 0)}")

print("\nSample items:")
for p, y in list(zip(file_paths, labels))[:5]:
    print(y, "->", CLASSES[y], "|", p)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

SAMPLE_RATE = 16000
FRAME_LENGTH = 512
FRAME_STEP   = 256
NUM_MELS     = 128
LOWER_HZ, UPPER_HZ = 80.0, 7600.0
TARGET_TIME_FRAMES = 61
TARGET_MEL_BINS    = 128
BATCH_SIZE = 16
SEED = 42

rng = np.random.default_rng(SEED)
perm = rng.permutation(len(file_paths))
file_paths = file_paths[perm]
labels     = labels[perm]

train_ratio = 0.8
train_size = int(len(file_paths) * train_ratio)

train_file_paths = file_paths[:train_size]
train_labels     = labels[:train_size]
valid_file_paths = file_paths[train_size:]
valid_labels     = labels[train_size:]

print(f"Train size: {len(train_file_paths)} | Valid size: {len(valid_file_paths)}")

@tf.function
def load_wav_16k_mono(file_path: tf.Tensor) -> tf.Tensor:
    audio_binary = tf.io.read_file(file_path)
    audio, _ = tf.audio.decode_wav(
        audio_binary, desired_channels=1, desired_samples=SAMPLE_RATE
    )
    return tf.squeeze(audio, axis=-1)

@tf.function
def wav_to_logmelspec(wav: tf.Tensor) -> tf.Tensor:
    stft = tf.signal.stft(wav, frame_length=FRAME_LENGTH, frame_step=FRAME_STEP)
    spectrogram = tf.abs(stft)

    num_spectrogram_bins = tf.shape(spectrogram)[-1]
    mel_w = tf.signal.linear_to_mel_weight_matrix(
        NUM_MELS, num_spectrogram_bins, SAMPLE_RATE, LOWER_HZ, UPPER_HZ
    )
    mel_spec = tf.tensordot(spectrogram, mel_w, axes=1)
    mel_spec.set_shape(spectrogram.shape[:-1].concatenate(mel_w.shape[-1:]))

    log_mel = tf.math.log(mel_spec + 1e-6)

    min_v = tf.reduce_min(log_mel)
    max_v = tf.reduce_max(log_mel)
    log_mel = (log_mel - min_v) / (max_v - min_v + 1e-9)

    log_mel = tf.expand_dims(log_mel, axis=-1)
    log_mel = tf.image.resize_with_pad(log_mel, TARGET_TIME_FRAMES, TARGET_MEL_BINS)
    return log_mel

@tf.function
def preprocess(file_path: tf.Tensor, label: tf.Tensor):
    wav = load_wav_16k_mono(file_path)
    spec = wav_to_logmelspec(wav)
    return spec, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((train_file_paths, train_labels))
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1000, seed=SEED, reshuffle_each_iteration=True)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

valid_ds = (
    tf.data.Dataset.from_tensor_slices((valid_file_paths, valid_labels))
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

for xb, yb in train_ds.take(1):
    print("Batch image shape:", xb.shape)
    print("Batch labels shape:", yb.shape)


# 2. CNN Model Training for Arabic **letters** classification

This block defines and trains a convolutional neural network (CNN) using TensorFlow/Keras for classifying Arabic letters based on log-mel spectrogram inputs.

- **Input setup:** Uses spectrogram tensors of shape **(61, 128, 1)** (time × mel bins × channel). Training runs for **100 epochs** (with early stopping).  
- **Custom metrics:** Implements `MicroPrecision`, `MicroRecall`, and `MicroF1Score` as Keras metrics. These are simplified approximations that track correct vs incorrect predictions across all classes and report micro-averaged precision, recall, and F1 in addition to standard accuracy.  
- **CNN architecture (Sequential):**  
  - Conv2D(32) → MaxPool(2×2)  
  - Conv2D(64) → MaxPool(2×2)  
  - Conv2D(128) → MaxPool(2×2)  
  - Conv2D(128) → MaxPool(2×2)  
  - Flatten  
  - Dense(512, ReLU) → Dropout(0.3)  
  - Dense(NUM_CLASSES, Softmax) for final classification  
- **Compilation:** Optimizer = **Adam**, loss = **sparse categorical crossentropy**, metrics = accuracy + custom precision/recall/F1.  
- **Callbacks:**  
  - `ModelCheckpoint` saves the best weights to `/content/model_hijaiyah_replica.h5` based on validation accuracy.  
  - `EarlyStopping` monitors validation accuracy with **patience=10** and restores best weights.  
- **Training:** Runs `model.fit()` on `train_ds`, validates on `valid_ds`, logs metrics each epoch, and saves the best-performing model.  
- **Output:** Prints model summary before training and confirms the path of the best saved checkpoint.  

**Purpose:** Builds a deep CNN to recognize 28 Arabic letters from spectrogram features, monitors training with both standard and custom evaluation metrics, and saves the best-performing model automatically.


In [ ]:
INPUT_SHAPE = (61, 128, 1)
EPOCHS = 100

class MicroPrecision(tf.keras.metrics.Metric):
    def __init__(self, name='micro_precision', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        y_true = tf.cast(y_true, tf.int64)
        y_pred = tf.cast(y_pred, tf.int64)
        correct = tf.cast(tf.equal(y_true, y_pred), tf.float32)
        self.tp.assign_add(tf.reduce_sum(correct))
        self.fp.assign_add(tf.reduce_sum(1.0 - correct))

    def result(self):
        return self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())

class MicroRecall(tf.keras.metrics.Metric):
    def __init__(self, name='micro_recall', **kwargs):
        super().__init__(name=name, **kwargs)
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        y_true = tf.cast(y_true, tf.int64)
        y_pred = tf.cast(y_pred, tf.int64)
        correct = tf.cast(tf.equal(y_true, y_pred), tf.float32)
        self.tp.assign_add(tf.reduce_sum(correct))
        self.fn.assign_add(tf.reduce_sum(
            tf.cast(tf.logical_and(tf.not_equal(y_true, y_pred), tf.equal(y_true, 0)), tf.float32)
        ))

    def result(self):
        return self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())

class MicroF1Score(tf.keras.metrics.Metric):
    def __init__(self, name='micro_f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.p = MicroPrecision()
        self.r = MicroRecall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.p.update_state(y_true, y_pred)
        self.r.update_state(y_true, y_pred)

    def result(self):
        p = self.p.result()
        r = self.r.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', MicroPrecision(), MicroRecall(), MicroF1Score()]
)

model.summary()

ckpt_path = "/content/model_hijaiyah_replica.h5"
cbs = [
    callbacks.ModelCheckpoint(ckpt_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True, verbose=1)
]

hist = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=cbs,
    verbose=1
)

print("Best model saved to:", ckpt_path)


# 3. Evaluation, diagnostics, and on-device inference utilities (Arabic **letters** model)

This block evaluates the trained CNN on the validation set, visualizes performance, computes Top-k metrics, and provides convenient **file-level prediction** utilities (including auto-conversion to 16 kHz mono WAV via `ffmpeg`) plus a Colab file-upload flow.

- **Validation metrics & reports**
  - Runs `model.evaluate(valid_ds)` and prints all compiled metrics.
  - Collects predictions over `valid_ds` to generate:
    - `classification_report` (per-class precision/recall/F1 using `CLASSES` names).
    - **Confusion matrix** (raw and row-normalized), with a heatmap (Matplotlib).

- **Top-k accuracy**
  - Computes **Top-1** (argmax) and **Top-5** accuracies from predicted probability vectors.
  - Utility `topk_acc()` provided for generic k.

- **Single-file prediction helpers**
  - `_load_wav_16k()` and `predict_file(path)` → fast path for already-16kHz mono WAV.
  - `convert_to_wav16k_mono(in_path)` uses **ffmpeg** to convert arbitrary audio (`.wav/.m4a/.mp3`) to 16 kHz mono WAV (saved in `/content/converted`).
  - `_load_wav_exact_1s()` pads/trims to exactly 1.0s (16,000 samples) for consistent features.
  - `predict_file_any_audio(in_path)` returns **top-1 label + confidence** and a **Top-5 dict**.

- **Interactive Colab upload**
  - Prompts to upload one or more audio files, auto-converts, plays back the converted WAV, and prints predictions with Top-5 probabilities.

- **Figures & history export**
  - Creates `/content/figures_letters/` and saves:
    - Accuracy and loss curves (`letters_acc_curve.png`, `letters_loss_curve.png`).
    - Confusion matrix (`letters_confusion_matrix.png`).
    - Raw Keras `history.history` as JSON (`letters_history.json`) for later analysis.

**Purpose:** End-to-end post-training analysis and demo tooling—quantitative metrics, visual diagnostics, reproducible figure exports, and quick ad-hoc inference on user-provided audio—so you can validate model quality and showcase predictions directly in Colab.


In [ ]:
results = model.evaluate(valid_ds, verbose=1)
print("\nMetrics on validation set:")
for name, val in zip(model.metrics_names, results):
    print(f"  {name:>22s}: {val:.4f}")

y_true, y_pred = [], []
for xb, yb in valid_ds:
    probs = model.predict(xb, verbose=0)
    y_pred.extend(np.argmax(probs, axis=1))
    y_true.extend(yb.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

all_labels = np.arange(NUM_CLASSES)

present = np.unique(y_true)
missing = sorted(set(all_labels) - set(present))
print("\nLabel diagnostics:")
print("  Present labels: ", present.tolist())
print("  Missing labels: ", missing)
if len(CLASSES) != NUM_CLASSES:
    print(f"⚠️  CLASSES length ({len(CLASSES)}) != NUM_CLASSES ({NUM_CLASSES})")

print("\nClassification report (per class):\n")
print(classification_report(
    y_true, y_pred,
    labels=all_labels,
    target_names=CLASSES,
    digits=3,
    zero_division=0
))

cm = confusion_matrix(y_true, y_pred, labels=all_labels)
cm_norm = cm / (cm.sum(axis=1, keepdims=True) + 1e-9)

plt.figure(figsize=(10, 9))
plt.imshow(cm_norm, interpolation='nearest', cmap='Blues', aspect='auto')
plt.title("Confusion Matrix (normalized)")
plt.colorbar()
ticks = np.arange(NUM_CLASSES)
plt.xticks(ticks, CLASSES, rotation=90)
plt.yticks(ticks, CLASSES)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

confusions = []
for i in range(NUM_CLASSES):
    for j in range(NUM_CLASSES):
        if i != j and cm[i, j] > 0:
            confusions.append((CLASSES[i], CLASSES[j], int(cm[i, j])))
confusions.sort(key=lambda x: x[2], reverse=True)

print("\nTop confusions (true -> predicted, count):")
for t, p, c in confusions[:20]:
    print(f"{t:>7s} -> {p:<7s} : {c}")


In [ ]:
top1_correct = 0
top5_correct = 0
n = 0

for xb, yb in valid_ds:
    probs = model.predict(xb, verbose=0)
    y_true = yb.numpy()
    top1 = np.argmax(probs, axis=1)
    top5 = np.argsort(-probs, axis=1)[:, :5]

    top1_correct += (top1 == y_true).sum()
    top5_correct += sum(y in top5[i] for i, y in enumerate(y_true))
    n += y_true.shape[0]

print(f"Top-1 accuracy: {top1_correct/n:.3f}")
print(f"Top-5 accuracy: {top5_correct/n:.3f}")


In [ ]:
def _load_wav_16k(path):
    audio_binary = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio_binary, desired_channels=1, desired_samples=16000)
    return tf.squeeze(audio, -1)

def predict_file(path, model=model):
    wav = _load_wav_16k(path)
    spec = wav_to_logmelspec(wav)
    spec = tf.expand_dims(spec, 0)
    probs = model.predict(spec, verbose=0)[0]
    top_idx = int(np.argmax(probs))
    top_label = CLASSES[top_idx]
    top_conf = float(probs[top_idx])

    order = np.argsort(-probs)[:5]
    top5 = {CLASSES[i]: float(probs[i]) for i in order}
    return top_label, top_conf, top5


In [ ]:
CONVERT_DIR = "/content/converted"
os.makedirs(CONVERT_DIR, exist_ok=True)

def convert_to_wav16k_mono(in_path: str) -> str:
    """Use ffmpeg to convert to 16 kHz mono WAV. Returns output path."""
    base = os.path.splitext(os.path.basename(in_path))[0]
    out_path = os.path.join(CONVERT_DIR, f"{base}_{uuid.uuid4().hex[:8]}.wav")
    cmd = ["ffmpeg", "-y", "-i", in_path, "-ac", "1", "-ar", "16000", out_path]
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if proc.returncode != 0 or not os.path.exists(out_path):
        raise RuntimeError(f"ffmpeg failed converting {in_path} -> WAV:\n{proc.stderr.decode(errors='ignore')}")
    return out_path

def _load_wav_exact_1s(path: str) -> tf.Tensor:
    """Load WAV as 16k mono and pad/trim to exactly 1.0 s (16000 samples)."""
    audio_binary = tf.io.read_file(path)
    audio, sr = tf.audio.decode_wav(audio_binary, desired_channels=1)
    wav = tf.squeeze(audio, -1)
    target = 16000
    n = tf.shape(wav)[0]
    def trim():
        return wav[:target]
    def pad():
        pad_len = target - n
        return tf.pad(wav, [[0, pad_len]])
    wav = tf.cond(n >= target, trim, pad)
    return wav

def predict_file_any_audio(in_path: str, model=model):
    """Convert arbitrary audio to 16k WAV, make log-mel, predict top-1 & top-5."""
    wav_path = convert_to_wav16k_mono(in_path)
    wav = _load_wav_exact_1s(wav_path)
    spec = wav_to_logmelspec(wav)
    spec = tf.expand_dims(spec, 0)
    probs = model.predict(spec, verbose=0)[0]
    top_idx = int(np.argmax(probs))
    top_label = CLASSES[top_idx]
    top_conf = float(probs[top_idx])
    order = np.argsort(-probs)[:5]
    top5 = {CLASSES[i]: float(probs[i]) for i in order}
    return top_label, top_conf, top5, wav_path


In [ ]:
print("📤 Select one or more audio files (.wav, .m4a, .mp3). I'll convert to 16 kHz mono WAV.")
uploaded = files.upload()

for fname, _ in uploaded.items():
    in_path = f"/content/{fname}"
    try:
        label, conf, top5, wav16_path = predict_file_any_audio(in_path)
        print("\n────────────")
        print(f"🎧 {fname}  →  converted: {os.path.basename(wav16_path)}")
        display(Audio(wav16_path))
        print(f"🔮 Prediction: {label}  ({conf:.2%})")
        print("🏅 Top‑5:")
        for k, v in top5.items():
            print(f"   - {k:10s}: {v:.2%}")
    except Exception as e:
        print(f"⚠️ Error on {fname}: {e}")


In [ ]:
FIG_DIR = "/content/figures_letters"
os.makedirs(FIG_DIR, exist_ok=True)

def to_numpy_labels(y):
    """Accept one-hot or integer labels -> return integer labels."""
    y = np.asarray(y)
    if y.ndim > 1 and y.shape[-1] > 1:
        return np.argmax(y, axis=-1)
    return y

def collect_from_dataset(ds):
    """Collect features and integer labels from a tf.data dataset."""
    ys_true, ys_prob = [], []
    for batch in ds:
        if isinstance(batch, (list, tuple)) and len(batch) == 2:
            x, y = batch
        else:
            raise ValueError("val_ds should yield (x, y)")
        probs = model.predict(x, verbose=0)
        ys_prob.append(probs)
        ys_true.append(y.numpy() if hasattr(y, "numpy") else y)
    y_true = to_numpy_labels(np.concatenate(ys_true, axis=0))
    y_prob = np.concatenate(ys_prob, axis=0)
    return y_true, y_prob

def plot_training_curves(history, title_prefix="Letters Model"):
    h = history.history
    plt.figure(figsize=(6,4))
    plt.plot(h.get("accuracy", []), label="train acc")
    plt.plot(h.get("val_accuracy", []), label="val acc")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()
    plt.title(f"{title_prefix}: Accuracy")
    acc_path = os.path.join(FIG_DIR, "letters_acc_curve.png")
    plt.tight_layout(); plt.savefig(acc_path, dpi=300); plt.show()
    plt.figure(figsize=(6,4))
    plt.plot(h.get("loss", []), label="train loss")
    plt.plot(h.get("val_loss", []), label="val loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
    plt.title(f"{title_prefix}: Loss")
    loss_path = os.path.join(FIG_DIR, "letters_loss_curve.png")
    plt.tight_layout(); plt.savefig(loss_path, dpi=300); plt.show()
    return acc_path, loss_path

def plot_confmat(cm, class_names, normalize=True, title="Letters Confusion Matrix"):
    if normalize:
        cm = cm.astype("float") / (cm.sum(axis=1, keepdims=True) + 1e-12)
    plt.figure(figsize=(8,6))
    im = plt.imshow(cm, interpolation="nearest", aspect="auto")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(ticks=np.arange(len(class_names)), labels=class_names, rotation=90)
    plt.yticks(ticks=np.arange(len(class_names)), labels=class_names)
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.title(title)
    plt.tight_layout()
    path = os.path.join(FIG_DIR, "letters_confusion_matrix.png")
    plt.savefig(path, dpi=300); plt.show()
    return path

def topk_accuracy(y_true, y_prob, k=5):
    """Compute Top-k accuracy given integer y_true and probability matrix y_prob."""
    topk = np.argsort(-y_prob, axis=1)[:, :k]
    match = (topk == y_true.reshape(-1,1)).any(axis=1)
    return match.mean()


In [ ]:
FIG_DIR = "/content/figures_letters"; os.makedirs(FIG_DIR, exist_ok=True)

def plot_and_save(history, key, title, fname):
    plt.figure(figsize=(6,4))
    plt.plot(history.history[key], label=f"train {key}")
    plt.plot(history.history[f"val_{key}"], label=f"val {key}")
    plt.xlabel("Epoch"); plt.ylabel(key.capitalize()); plt.legend()
    plt.title(title); plt.tight_layout()
    path = os.path.join(FIG_DIR, fname); plt.savefig(path, dpi=300); plt.show()
    return path

acc_png = plot_and_save(hist, "accuracy", "Letters model: accuracy", "letters_acc_curve.png")
loss_png = plot_and_save(hist, "loss", "Letters model: loss", "letters_loss_curve.png")
print("Saved:", acc_png, loss_png)

import json, pathlib
with open(pathlib.Path(FIG_DIR)/"letters_history.json","w") as f:
    json.dump(hist.history, f)


In [ ]:
y_true, y_prob = [], []
for x_batch, y_batch in valid_ds:
    probs = model.predict(x_batch, verbose=0)
    y_prob.append(probs)
    y_true.append(y_batch.numpy())
y_prob = np.concatenate(y_prob, axis=0)
y_true = np.concatenate(y_true, axis=0)
y_pred = np.argmax(y_prob, axis=1)

def topk_acc(y_true, y_prob, k=5):
    topk = np.argsort(-y_prob, axis=1)[:, :k]
    return np.mean([yt in row for yt, row in zip(y_true, topk)])

top1 = accuracy_score(y_true, y_pred)
top5 = topk_acc(y_true, y_prob, k=5)
print(f"Top-1: {top1:.3f} | Top-5: {top5:.3f}")
